In [1]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv('out_parsed', parse_dates=['actual_date', 'eta'])

In [3]:
df['remaining_seconds'] = pd.to_timedelta(df['eta'] - df['actual_date'], 's')
df['eta_date'] = df['eta'].dt.day
df.head()

,actual_date,cod_stop,cod_line,cod_issue,eta,destination_stop,remaining_seconds,eta_date
0,2020-02-15 17:45:40+01:00,8_23,8__629___,8__629____2_17:05:00_2_-__13_8__629___,2020-02-15 17:46:34+01:00,8_06002,00:00:54,15
1,2020-02-15 17:45:40+01:00,8_23,8__686___,5278458,2020-02-15 17:47:01+01:00,8_17480,00:01:21,15
2,2020-02-15 17:45:40+01:00,8_23,8__651___,5308556,2020-02-15 17:47:19+01:00,8_17480,00:01:39,15
3,2020-02-15 17:45:40+01:00,8_23,8__654___,5308548,2020-02-15 17:48:21+01:00,8_17480,00:02:41,15
4,2020-02-15 17:45:40+01:00,8_23,8__622___,8__622____2_17:00:00_2_-__17_8__622___,2020-02-15 17:48:56+01:00,8_06002,00:03:16,15


In [4]:
arrival_times_rows = []
#columns = ['arrival_time', 'cod_issue', 'cod_stop', 'cod_line']
df_grouped = df.groupby(['cod_issue', 'cod_stop', 'cod_line', 'eta_date'])
arrival_times = (df_grouped.apply(lambda x: x.sort_values('remaining_seconds', ascending=True).iloc[0]['eta'])
                           .to_frame(name = 'arrival_time')
                           .reset_index()
                           .sort_values('arrival_time', ascending=True)
                )

In [5]:
arrival_times

,cod_issue,cod_stop,cod_line,eta_date,arrival_time
598256,8__622____2_17:00:00_2_-__17_8__622___,8_4311,8__622___,15,2020-02-15 17:45:53+01:00
467194,5308274,8_10577,8__562___,15,2020-02-15 17:46:15+01:00
473742,5308413,8_1333,8__653___,15,2020-02-15 17:46:15+01:00
637226,8__661____2_17:45:00_1_-__21_8__661___,8_1338,8__661___,15,2020-02-15 17:46:20+01:00
472226,5308386,8_08458,8__564___,15,2020-02-15 17:46:27+01:00
...,...,...,...,...,...
637525,8__662____1_07:30:00_1_-__2_8__662___,8_1338,8__662___,3,2020-03-03 07:31:20+01:00
638141,8__664____1_08:05:00_1_-__3_8__664___,8_1338,8__664___,3,2020-03-03 08:06:25+01:00
637548,8__662____1_08:25:00_1_-__3_8__662___,8_1338,8__662___,3,2020-03-03 08:26:26+01:00
634629,8__661_A___1_09:10:00_1_-__2_8__661_A__,8_06002,8__661_A__,3,2020-03-03 09:10:00+01:00


In [6]:
arrival_times.to_csv('arrival_times.csv', index=False)

In [15]:
import plotly.express as px

In [74]:
test_df = df
#test_df = df.query("cod_issue == '5302729' & eta_date == 2 & cod_stop == '8_06277'")
test_df = test_df[test_df['remaining_seconds'] < pd.Timedelta(1, unit='m')]
#fig = px.scatter(test_df, x='actual_date', y='remaining_seconds')
#fig.show()

In [75]:
test_arrival_times_rows = []
#columns = ['arrival_time', 'cod_issue', 'cod_stop', 'cod_line']
#test_df['ad_plus_rs'] = test_df['actual_date'] + test_df['remaining_seconds']
test_df_grouped = test_df.groupby(['cod_issue', 'cod_stop', 'cod_line', 'eta_date'])
test_arrival_times = (test_df_grouped.apply(lambda x: x.sort_values(['actual_date'], ascending=[True]).iloc[0]['eta'])
                           .to_frame(name = 'arrival_time')
                           .reset_index()
                           .sort_values('arrival_time', ascending=True)
                )
test_arrival_times

,cod_issue,cod_stop,cod_line,eta_date,arrival_time
475354,8__622____2_17:00:00_2_-__17_8__622___,8_4311,8__622___,15,2020-02-15 17:45:53+01:00
387318,5308274,8_10577,8__562___,15,2020-02-15 17:46:15+01:00
392837,5308413,8_1333,8__653___,15,2020-02-15 17:46:15+01:00
503011,8__661____2_17:45:00_1_-__21_8__661___,8_1338,8__661___,15,2020-02-15 17:46:20+01:00
391605,5308386,8_08458,8__564___,15,2020-02-15 17:46:27+01:00
...,...,...,...,...,...
70273,5302610,8_10580,8__564___,2,2020-03-02 19:01:22+01:00
479880,8__625____1_18:12:00_2_-__60_8__625___,8_06268,8__625___,2,2020-03-02 19:01:27+01:00
67913,5302555,8_06296,8__563___,2,2020-03-02 19:01:29+01:00
493288,8__629____1_18:10:00_2_-__36_8__629___,8_1335,8__629___,2,2020-03-02 19:01:30+01:00


In [76]:
test_arrival_times.to_csv('arrival_times.csv', index=False)